# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

610

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
# create empty lists to store data for dataframe
city_list = []
latitude = []
longitude = []
temp = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = []

In [4]:
#create query url for loop
units = 'imperial'
query_url = f'http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={units}&q='

In [5]:
test_response = requests.get(query_url + 'houston').json()
test_response['main']['temp_max']

91.96

In [6]:
print('Beginning Data Retrieval')
print('-----------------------------')

#create query url for loop
query_url = f'http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units={units}&q='

# loop through cities to collect currect weather data
for city in cities:
    # Create search query, make request and store in json
    response_json = requests.get(query_url + city).json()
    
    # attempt to grab weather data from the cities
    try:
        latitude.append(response_json['coord']['lat'])
        longitude.append(response_json['coord']['lon'])
        temp.append(response_json['main']['temp'])
        max_temp.append(response_json['main']['temp_max'])
        humidity.append(response_json['main']['humidity'])
        cloudiness.append(response_json['clouds']['all'])
        wind_speed.append(response_json['wind']['speed'])
        city_list.append(city)
        #mass.append(response_json['results'][0]['mass'])
        print(f'Processing Record {cities.index(city)} of Set 1 | {city}')
        
    # exceptions for a cities not found or errors
    except:
        print(f'{city} not found. Skipping.')
        pass

Beginning Data Retrieval
-----------------------------
Processing Record 0 of Set 1 | mataura
Processing Record 1 of Set 1 | kazanskoye
Processing Record 2 of Set 1 | hobart
Processing Record 3 of Set 1 | saskylakh
Processing Record 4 of Set 1 | hermanus
Processing Record 5 of Set 1 | cape town
Processing Record 6 of Set 1 | hilo
Processing Record 7 of Set 1 | yellowknife
Processing Record 8 of Set 1 | punta arenas
Processing Record 9 of Set 1 | twentynine palms
Processing Record 10 of Set 1 | nadvoitsy
Processing Record 11 of Set 1 | hithadhoo
taolanaro not found. Skipping.
Processing Record 13 of Set 1 | busselton
Processing Record 14 of Set 1 | carnarvon
belushya guba not found. Skipping.
Processing Record 16 of Set 1 | nam som
Processing Record 17 of Set 1 | airai
Processing Record 18 of Set 1 | husavik
Processing Record 19 of Set 1 | foumban
Processing Record 20 of Set 1 | dukat
Processing Record 21 of Set 1 | bethel
barentsburg not found. Skipping.
Processing Record 23 of Set 1 |

Processing Record 201 of Set 1 | lokosovo
Processing Record 202 of Set 1 | oranjemund
Processing Record 203 of Set 1 | lavrentiya
louisbourg not found. Skipping.
Processing Record 205 of Set 1 | thompson
Processing Record 206 of Set 1 | ancud
Processing Record 207 of Set 1 | samarai
Processing Record 208 of Set 1 | jizan
Processing Record 209 of Set 1 | victoria
Processing Record 210 of Set 1 | fortuna
viligili not found. Skipping.
Processing Record 212 of Set 1 | dryden
Processing Record 213 of Set 1 | ritchie
Processing Record 214 of Set 1 | ponta do sol
Processing Record 215 of Set 1 | taboshar
Processing Record 216 of Set 1 | lebu
Processing Record 217 of Set 1 | gubkinskiy
Processing Record 218 of Set 1 | talpa
Processing Record 219 of Set 1 | mahebourg
Processing Record 220 of Set 1 | xiuyan
Processing Record 221 of Set 1 | balclutha
Processing Record 222 of Set 1 | comodoro rivadavia
Processing Record 223 of Set 1 | kudahuvadhoo
Processing Record 224 of Set 1 | cabo san lucas
Pr

Processing Record 402 of Set 1 | aparecida do taboado
Processing Record 403 of Set 1 | zambezi
Processing Record 404 of Set 1 | ponta delgada
Processing Record 405 of Set 1 | kruisfontein
Processing Record 406 of Set 1 | kutum
Processing Record 407 of Set 1 | hamilton
Processing Record 408 of Set 1 | antequera
Processing Record 409 of Set 1 | abu dhabi
Processing Record 410 of Set 1 | san patricio
Processing Record 411 of Set 1 | taltal
Processing Record 412 of Set 1 | port blair
Processing Record 413 of Set 1 | half moon bay
Processing Record 414 of Set 1 | komsomolskiy
Processing Record 415 of Set 1 | chatellerault
Processing Record 416 of Set 1 | yeppoon
Processing Record 417 of Set 1 | hornepayne
Processing Record 418 of Set 1 | tumut
Processing Record 419 of Set 1 | yarmouth
Processing Record 420 of Set 1 | torquay
Processing Record 421 of Set 1 | tara
Processing Record 422 of Set 1 | mogadishu
Processing Record 423 of Set 1 | port keats
Processing Record 424 of Set 1 | yeroham
Pr

Processing Record 600 of Set 1 | kidal
Processing Record 601 of Set 1 | rancho palos verdes
Processing Record 602 of Set 1 | le port
Processing Record 603 of Set 1 | sterlibashevo
Processing Record 604 of Set 1 | oktyabrskoye
Processing Record 605 of Set 1 | marystown
Processing Record 606 of Set 1 | tawang
Processing Record 607 of Set 1 | rapid valley
Processing Record 608 of Set 1 | mwinilunga
duz not found. Skipping.


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [9]:
# Create DataFrame
weather_data = pd.DataFrame({
    'City': city_list,
    'Latitude': latitude,
    'Longitude': longitude,
    'Temperature (F)': temp,
    'Max Temp (F)': max_temp,
    'Humidity (%)': humidity,
    'Cloudiness (%)': cloudiness,
    'Wind Speed (mph)': wind_speed
})

#set the city to the index
weather_data = weather_data.set_index('City')

weather_data

,Latitude,Longitude,Temperature (F),Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
City,,,,,,,
mataura,-46.1927,168.8643,32.49,32.49,99,99,3.76
kazanskoye,55.6448,69.2350,64.80,64.80,74,91,11.21
hobart,-42.8794,147.3294,44.08,46.26,81,11,1.01
saskylakh,71.9167,114.0833,59.07,59.07,61,95,10.80
hermanus,-34.4187,19.2345,63.81,63.81,70,100,25.50
...,...,...,...,...,...,...,...
oktyabrskoye,43.0645,44.7417,78.94,78.94,74,75,6.71
marystown,47.1666,-55.1483,68.25,68.25,80,100,12.30
tawang,27.5833,91.8667,59.02,59.02,98,100,1.90


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [10]:
clean_weather_data = weather_data.loc[(weather_data['Humidity (%)'] < 100),:]

#export clean data to csv
clean_weather_data.to_csv('../Output/clean_weather_data.csv')

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
clean_weather_data.plot(kind='scatter', x='Latitude', y='Max Temp (F)', grid=True, figsize=(9,6),
              title='City Latitude vs. Temperature', s=25, alpha=0.8, xlim=(-60,90))

plt.savefig('../Output/lat_v_temp', pad_inches=0.1, facecolor='w')
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
clean_weather_data.plot(kind='scatter', x='Latitude', y='Humidity (%)', grid=True, figsize=(9,6),
              title='City Latitude vs. Humidity', s=25, alpha=0.8, xlim=(-60,90))

plt.savefig('../Output/lat_v_humidity', pad_inches=0.1, facecolor='w')
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
clean_weather_data.plot(kind='scatter', x='Latitude', y='Cloudiness (%)', grid=True, figsize=(9,6),
              title='City Latitude vs. Cloudiness', s=25, alpha=0.8, xlim=(-60,90))

plt.savefig('../Output/lat_v_clouds', pad_inches=0.1, facecolor='w')
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
clean_weather_data.plot(kind='scatter', x='Latitude', y='Wind Speed (mph)', grid=True, figsize=(9,6),
              title='City Latitude vs. Wind Speed', s=25, alpha=0.8, xlim=(-60,90))

plt.savefig('../Output/lat_v_wind', pad_inches=0.1, facecolor='w')
plt.show()

## Linear Regression

In [ ]:
#create dataframes for Northen and Southern Hemispheres
northern_hemi = clean_weather_data.loc[(clean_weather_data['Latitude'] > 0),:]
southern_hemi = clean_weather_data.loc[(clean_weather_data['Latitude'] < 0),:]


####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
northern_hemi.plot(kind='scatter', x='Latitude', y='Max Temp (F)', grid=True, figsize=(9,6),
              title='City Latitude vs. Temperature', s=25, alpha=0.8, xlim=(-60,90))

#plt.savefig('../Output/lat_v_temp', pad_inches=0.1, facecolor='w')
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
southern_hemi.plot(kind='scatter', x='Latitude', y='Max Temp (F)', grid=True, figsize=(9,6),
              title='City Latitude vs. Temperature', s=25, alpha=0.8, xlim=(-60,90))

plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression